In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movies/tmdb_5000_movies.csv
/kaggle/input/credits/tmdb_5000_credits.csv


# **make dataframe by pandas**

In [2]:
movies = pd.read_csv("/kaggle/input/movies/tmdb_5000_movies.csv")
credits = pd.read_csv("/kaggle/input/credits/tmdb_5000_credits.csv")

movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


# **observe and understand the dataset**

In [3]:
movies.head(1)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [4]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [5]:
credits.head(1)["cast"]

0    [{"cast_id": 242, "character": "Jake Sully", "...
Name: cast, dtype: object

In [6]:
credits.head(1)["crew"]

0    [{"credit_id": "52fe48009251416c750aca23", "de...
Name: crew, dtype: object

In [7]:
movies.shape

(4803, 20)

In [8]:
credits.shape

(4803, 4)

# **merge two dataframes into a single dataframe**

In [9]:
movies = movies.merge(credits, on="title")

In [10]:
movies.shape

(4809, 23)

In [11]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


# **what columns I would keep**



1. genre
2. id
3. keyword
4. title
5. overview
6. cast
7. crew


In [12]:
movies = movies[["id","title","overview", "genres","keywords","cast","crew"]]

In [13]:
movies.head(1)
movies.shape

(4809, 7)

In [14]:
#movies.head(1)["crew"].values


In [15]:
import json


crew_json = json.loads(movies.loc[0, "crew"])


departments = [person['department'] for person in crew_json]


unique_departments = set(departments)
print(unique_departments)

{'Editing', 'Art', 'Lighting', 'Production', 'Sound', 'Visual Effects', 'Directing', 'Writing', 'Costume & Make-Up', 'Crew', 'Camera'}


# **data preprocessing start**
1. genre, overview, cast and crew will merge
2. keep 2/3 importat cast
3. keep only directing crew
4. null value handle
5. duplicate value handle

In [16]:
movies.isnull().sum()
# 3 row in overview has null values.

id          0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [17]:
movies.dropna(inplace=True)

In [18]:
movies.isnull().sum()

id          0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [19]:
movies.duplicated().sum()

# no duplicated row. 

0

# **Now start merge of genres, overview, cast and crew in a single tagline**

In [20]:
movies.iloc[5].genres

'[{"id": 14, "name": "Fantasy"}, {"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}]'

In [21]:
# '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
# should convert like the following:
# ["action", "adventure", "fantasy", "science fiction"]

# first ready genres column. make it a simple array 


In [22]:
import ast

def convertion(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i["name"])
    return l    


In [23]:
# for checking purpose

p = movies.iloc[5].genres
a = convertion(p)
print(a)

['Fantasy', 'Action', 'Adventure']


In [24]:
movies["genres"] = movies["genres"].apply(convertion)

In [25]:
movies.head(2)

# genres of the dataframe is in the perfect format.

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


# **now have to do the same thing for keywords column**

In [26]:
movies["keywords"] = movies["keywords"].apply(convertion)

In [27]:
movies.head(2)

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [28]:
movies.iloc[5]["keywords"]

['dual identity',
 'amnesia',
 'sandstorm',
 "love of one's life",
 'forgiveness',
 'spider',
 'wretch',
 'death of a friend',
 'egomania',
 'sand',
 'narcism',
 'hostility',
 'marvel comic',
 'sequel',
 'superhero',
 'revenge']

# now keep only first 3 casts

In [29]:
movies.iloc[5]["cast"]

'[{"cast_id": 30, "character": "Peter Parker / Spider-Man", "credit_id": "52fe4252c3a36847f80151c7", "gender": 2, "id": 2219, "name": "Tobey Maguire", "order": 0}, {"cast_id": 7, "character": "Mary Jane Watson", "credit_id": "52fe4252c3a36847f801514f", "gender": 1, "id": 205, "name": "Kirsten Dunst", "order": 1}, {"cast_id": 35, "character": "Harry Osborn / New Goblin", "credit_id": "52fe4252c3a36847f80151db", "gender": 2, "id": 17051, "name": "James Franco", "order": 2}, {"cast_id": 36, "character": "Flint Marko / Sandman", "credit_id": "52fe4252c3a36847f80151df", "gender": 2, "id": 19159, "name": "Thomas Haden Church", "order": 3}, {"cast_id": 37, "character": "Eddie Brock / Venom", "credit_id": "52fe4252c3a36847f80151e3", "gender": 2, "id": 17052, "name": "Topher Grace", "order": 4}, {"cast_id": 8, "character": "Gwen Stacey", "credit_id": "52fe4252c3a36847f8015153", "gender": 1, "id": 18997, "name": "Bryce Dallas Howard", "order": 5}, {"cast_id": 31, "character": "May Parker", "cred

In [30]:
def convertion2(obj):
    l = []
    count = 0
    for i in ast.literal_eval(obj):
        if count < 3:
            l.append(i["name"])
            count += 1
        else:
            break
        
    return l    

In [31]:
test = movies.iloc[1]["cast"]
# print(test)
ans = convertion2(test)
print(ans)

['Johnny Depp', 'Orlando Bloom', 'Keira Knightley']


In [32]:
movies["cast"] = movies["cast"].apply(convertion2)

In [33]:
movies.head(3)
# movies.shape

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


# now have to ready crew. In crew i keep only director name

In [34]:
def fetch_director(obj):
    l = []
    for i in ast.literal_eval(obj):
        if i["job"] == "Director":
            l.append(i["name"])
            break
    return l        

In [35]:
t = movies.iloc[0]["crew"]
ans = fetch_director(t)
print(ans)

['James Cameron']


In [36]:
movies["crew"] = movies["crew"].apply(fetch_director)

In [37]:
movies.head(3)

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]


# create list of overview. right now it is string

In [38]:
movies.iloc[0]["overview"]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [39]:
movies["overview"] = movies["overview"].apply(lambda x: x.split())

In [40]:
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


# have to remove space between words

In [41]:
movies["cast"] = movies["cast"].apply(lambda x: [i.replace(" ","") for i in x])
movies["genres"] = movies["genres"].apply(lambda x: [i.replace(" ", "") for i in x])
movies["crew"] = movies["crew"].apply(lambda x: [i.replace(" ", "") for i in x])
movies["keywords"] = movies["keywords"].apply(lambda x: [i.replace(" ", "") for i in x])

In [42]:
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


# Make a new col "tags". it will store genres, keywords, overview, cast and crew


In [43]:
movies["tags"] = movies["overview"] + movies["genres"] + movies["keywords"] + movies["cast"] + movies["crew"]

In [44]:
movies.head(3)

,id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[A, cryptic, message, from, Bond’s, past, send..."


In [45]:
movies.iloc[2]["tags"]

['A',
 'cryptic',
 'message',
 'from',
 'Bond’s',
 'past',
 'sends',
 'him',
 'on',
 'a',
 'trail',
 'to',
 'uncover',
 'a',
 'sinister',
 'organization.',
 'While',
 'M',
 'battles',
 'political',
 'forces',
 'to',
 'keep',
 'the',
 'secret',
 'service',
 'alive,',
 'Bond',
 'peels',
 'back',
 'the',
 'layers',
 'of',
 'deceit',
 'to',
 'reveal',
 'the',
 'terrible',
 'truth',
 'behind',
 'SPECTRE.',
 'Action',
 'Adventure',
 'Crime',
 'spy',
 'basedonnovel',
 'secretagent',
 'sequel',
 'mi6',
 'britishsecretservice',
 'unitedkingdom',
 'DanielCraig',
 'ChristophWaltz',
 'LéaSeydoux',
 'SamMendes']

# create a total new dataframe

In [46]:
newdata = movies[["id", "title", "tags"]]

In [47]:
newdata.head(3)

,id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."


In [48]:
newdata = newdata.rename(columns={"id" : "movie_id"})

In [49]:
newdata.head(3)

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."


# convert tags in string

In [50]:
newdata["tags"] = newdata["tags"].apply(lambda x: " ".join(x))

In [51]:
newdata.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [52]:
newdata.iloc[0]["tags"]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

# lowercase convertion

In [53]:
newdata["tags"] = newdata["tags"].apply(lambda x: x.lower())

In [54]:
newdata.iloc[0]["tags"]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

# have to do stemming by nltk

In [55]:
import nltk
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

In [56]:
# make a function who will do stem

def stemming(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))

    return " ".join(y)

In [57]:
d = "i am nabil adnan. I am making a movie recommnder project"

stemming(d)

'i am nabil adnan. i am make a movi recommnd project'

In [58]:
newdata["tags"] = newdata["tags"].apply(stemming)

In [85]:
newdata.head(10)

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."
5,559,Spider-Man 3,the seemingli invinc spider-man goe up against...
6,38757,Tangled,when the kingdom' most wanted-and most charmin...
7,99861,Avengers: Age of Ultron,when toni stark tri to jumpstart a dormant pea...
8,767,Harry Potter and the Half-Blood Prince,"as harri begin hi sixth year at hogwarts, he d..."
9,209112,Batman v Superman: Dawn of Justice,fear the action of a god-lik super hero left u...


# Now I am picking 5000 words which are most common in all the tags of all the movies. Then i will do  text vectorization using bag of words technique. 

In [60]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words="english", max_features=5000)

In [61]:
type(newdata.iloc[0]["tags"])

str

In [62]:
newdata

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."
...,...,...,...
4804,9367,El Mariachi,el mariachi just want to play hi guitar and ca...
4805,72766,Newlyweds,a newlyw couple' honeymoon is upend by the arr...
4806,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduc a dedic q..."
4807,126186,Shanghai Calling,when ambiti new york attorney sam is sent to s...


In [63]:
cv.fit_transform(newdata["tags"]).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [64]:
cv.fit_transform(newdata["tags"]).toarray().shape

(4806, 5000)

# take vector from tags column by CountVectorizer

In [65]:
tags_vector = cv.fit_transform(newdata["tags"]).toarray()

In [66]:
# tags for first row. most are zero. we can say sparse matrix.
tags_vector[0]

array([0, 0, 0, ..., 0, 0, 0])

In [67]:
len(cv.get_feature_names_out())

5000

In [68]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

# now distance and smilarity measurement

In [69]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(tags_vector)

In [70]:
# similarity of first movie with every other movies
similarity[0]

array([1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
       0.        ])

In [71]:
sorted(list(enumerate(similarity[0])), reverse=True, key = lambda x: x[1])[1:6]
# enumerate korar karone index hishab korte parbo
# reverse diye descending order e sort hobe
# key diye second index ba similarity er basis e sorting hobe
# first 5 ta movie recommend e dekhabe

[(1214, 0.2867696673382022),
 (2405, 0.26901379342448517),
 (3728, 0.2605130246476754),
 (507, 0.255608593705383),
 (539, 0.25038669783359574)]

# finally recommendation by highest similarity

In [72]:
newdata[newdata["title"] == "Spectre"].index

# this index store all the matching values. so index[0] will gives only the first match.
#because it is itself the first match of it.

Index([2], dtype='int64')

In [78]:
def rec(movie):
    movie_index = newdata[newdata["title"] == movie].index[0]
    distances = similarity[movie_index] 
    # distances er moddhe ekhon oi movie er sathe onno shob movie gular distances gula ase.
    movie_list = sorted(list(enumerate(distances)), reverse=True, key = lambda x: x[1])[1:6]
    
    
    for i in movie_list:
        print(f"{newdata.iloc[i[0]].title} = {i[1]}") 

In [79]:
rec("Newlyweds")

Bad Grandpa = 0.26687249808205815
How to Fall in Love = 0.2346903478277816
The R.M. = 0.22421317963806006
Time Changer = 0.2114774672189068
After Earth = 0.20736758755125534


In [80]:
rec("Avatar")

Aliens vs Predator: Requiem = 0.2867696673382022
Aliens = 0.26901379342448517
Falcon Rising = 0.2605130246476754
Independence Day = 0.255608593705383
Titan A.E. = 0.25038669783359574


In [81]:
rec("Batman")

Batman = 0.942809041582063
Batman & Robin = 0.41147559989891186
Batman Begins = 0.3340765523905305
Batman Returns = 0.3250056229832958
The R.M. = 0.32242634688922367


In [82]:
rec("Batman Returns")

The Dark Knight = 0.3314209059996644
Batman = 0.3250056229832958
The Dark Knight Rises = 0.3247027084163318
Batman = 0.3041651646201797
Batman Forever = 0.2802759273356052


In [86]:
rec("The Dark Knight Rises")

The Dark Knight = 0.42256142372325745
Batman Returns = 0.3247027084163318
Batman = 0.3202563076101742
Batman Forever = 0.3181582090733562
Batman Begins = 0.3177444546511212


In [89]:
rec("Avengers: Age of Ultron")

Iron Man 3 = 0.35478743759344966
Iron Man 2 = 0.3474411464169823
Iron Man = 0.3359355065735125
Thor = 0.32304786919046097
The Avengers = 0.317925392228937


In [90]:
rec("Thor")

Thor: The Dark World = 0.3757691138408167
Clash of the Titans = 0.35816181173029066
After Earth = 0.33301229566534774
Ant-Man = 0.32991443953692906
Iron Man 2 = 0.32923769125601826


In [92]:
rec("Ant-Man")

Iron Man 2 = 0.44353276257274377
Krrish = 0.36647045615996826
Khumba = 0.36596252735569995
All Is Lost = 0.3646624787447365
1982 = 0.3572478837168502


In [93]:
rec("Khumba")

The Peanuts Movie = 0.5714285714285714
Should've Been Romeo = 0.5155392622264671
Flying By = 0.5111012519999519
The Diving Bell and the Butterfly = 0.4988876515698588
The Adventures of Elmo in Grouchland = 0.49801192055599736


In [95]:
rec("The Diving Bell and the Butterfly")

Revolutionary Road = 0.20343336780168372
Bound = 0.1898315991504998
Divine Secrets of the Ya-Ya Sisterhood = 0.18857857160117855
Dry Spell = 0.17574991006549875
Four Single Fathers = 0.17139779254776524


In [96]:
rec("Dry Spell")

I Served the King of England = 0.36099410255771214
Should've Been Romeo = 0.35137019269127184
The Truman Show = 0.3454937455142388
What's Eating Gilbert Grape = 0.3437619363855793
Little Boy = 0.3363363969981562
